In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
data=pd.read_csv('ecg.csv',header=None)
data.dropna(inplace=True)

In [ ]:
features = data.drop(140, axis=1)
target = data[140]
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, stratify=target
)



In [ ]:
train_index=y_train[y_train==1].index
train_data=x_train.loc[train_index]

In [ ]:
train_data

In [ ]:
mm=MinMaxScaler()
x_train_scaled=mm.fit_transform(train_data.copy())
x_test_Scaled=mm.fit_transform(x_test.copy())

In [ ]:
ldim=32
output_units=x_train_scaled.shape[1]
encoder = Sequential([
      Dense(64, activation='relu'),
      Dropout(0.1),
      Dense(32, activation='relu'),
      Dropout(0.1),
      Dense(16, activation='relu'),
      Dropout(0.1),
      Dense(ldim, activation='relu')
    ])
decoder = Sequential([
      Dense(16, activation='relu'),
      Dropout(0.1),
      Dense(32, activation='relu'),
      Dropout(0.1),
      Dense(64, activation='relu'),
      Dropout(0.1),
      Dense(output_units, activation='sigmoid')
    ])

In [ ]:
model = Sequential([
    encoder,
    decoder,
])

In [ ]:
model.compile(loss='msle', metrics=['mse'], optimizer='adam')

In [ ]:
epochs = 20

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=epochs,
    batch_size=512,
    validation_data=(x_test_Scaled, x_test_Scaled)
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('MSLE Loss')
plt.legend()
plt.show()

In [ ]:
decoded_data = model.predict(x_test_Scaled)
mse = np.mean(np.power(x_test_Scaled - decoded_data, 2), axis=1)
threshold = np.percentile(mse, 95)

In [ ]:
outliers = mse > threshold

In [ ]:
num_outliers = np.sum(outliers)
num_anomalies = np.sum(y_test[outliers] == 1)  # Assuming 1 represents the positive class

print(f'Number of outliers: {num_outliers}')
print(f'Number of anomalies: {num_anomalies}')